Парсер для получения комментариев по дочерним карточкам.
Пример: есть клиент ВШЭ, хочется узнать, что происходило с ним, но иннформация разбросана по дочерним карточкам (в случае с ВШЭ их сотни)

In [ ]:
import requests


API_KEY = "your_api_key"
BASE_URL = "https://proctoredu.kaiten.ru/api/latest"
CARD_ID = 3047259

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Accept": "application/json"
}

def get_card_data(card_id):
    response = requests.get(f"{BASE_URL}/cards/{card_id}", headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print("Ошибка запроса карточки:", response.status_code, response.text)
        return None

def get_card_comments(card_id):
    response = requests.get(f"{BASE_URL}/cards/{card_id}/comments", headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print("Не удалось получить комментарии:", response.status_code, response.text)
        return []


def get_child_cards(card_id):
    response = requests.get(f"{BASE_URL}/cards/{card_id}/children", headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print("Не удалось получить дочерние карточки:", response.status_code, response.text)
        return []

# Основной блок кода
with open("card_data.txt", "w", encoding="utf-8") as file:
    card_data = get_card_data(CARD_ID)
    if card_data:
        file.write(f"ID: {card_data.get('id')}\n")
        file.write(f"Название: {card_data.get('title')}\n")
        file.write(f"Описание: {card_data.get('description')}\n")
        file.write(f"Дата создания: {card_data.get('created')}\n")
        file.write(f"Дата обновления: {card_data.get('updated')}\n\n")

        comments = get_card_comments(CARD_ID)
        file.write("Комментарии:\n")
        for comment in comments:
            file.write(f"- {comment.get('text')} (Автор: {comment.get('author_id')}, Дата: {comment.get('created')})\n")
        file.write("\n")

        child_cards = get_child_cards(CARD_ID)
        file.write("Дочерние карточки:\n")
        for child in child_cards:
            child_data = get_card_data(child.get("id"))
            if child_data:
                file.write(f"- {child_data.get('title')} (ID: {child_data.get('id')})\n")
                file.write(f"  Описание: {child_data.get('description')}\n")
                file.write(f"  Дата создания: {child_data.get('created')}\n")
                file.write(f"  Дата обновления: {child_data.get('updated')}\n")

                child_comments = get_card_comments(child_data.get("id"))
                file.write("  Комментарии:\n")
                for comment in child_comments:
                    file.write(f"  - {comment.get('text')} (Автор: {comment.get('author_id')}, Дата: {comment.get('created')})\n")
                file.write("\n")

print("Данные сохранены в card_data.txt")
#по вшэ ошибки -- это норм, там часть картчоек удалили, всё работает корректно

Ошибка запроса карточки: 403 {"message":"Card with id 5084451 became not accessible"}
Ошибка запроса карточки: 403 {"message":"Card with id 5112266 became not accessible"}
Ошибка запроса карточки: 403 {"message":"Card with id 5261221 became not accessible"}
Ошибка запроса карточки: 403 {"message":"Card with id 5562678 became not accessible"}
Ошибка запроса карточки: 403 {"message":"Card with id 6619211 became not accessible"}
Ошибка запроса карточки: 403 {"message":"Card with id 8531271 became not accessible"}
Ошибка запроса карточки: 403 {"message":"Card with id 8614312 became not accessible"}
Ошибка запроса карточки: 403 {"message":"Card with id 8642902 became not accessible"}
Ошибка запроса карточки: 403 {"message":"Card with id 8667794 became not accessible"}
Ошибка запроса карточки: 403 {"message":"Card with id 8707102 became not accessible"}
Ошибка запроса карточки: 403 {"message":"Card with id 8746735 became not accessible"}
Ошибка запроса карточки: 403 {"message":"Card with id 

Парсер для сбора карточек по определённому типу на определённом пространстве. Есть ограничение: не больше 100 карточек за раз. Пример: собираем карточки с тегом "Новый лид".

In [ ]:
import requests
import csv
import re

API_KEY = "your_api_key"
BASE_URL = "https://proctoredu.kaiten.ru/api/latest"


headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Accept": "application/json"
}

def get_cards_by_tag(space_id, tag, limit=100):
    url = f"{BASE_URL}/cards?space_id={space_id}&tag={tag}&limit={limit}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print("Ошибка запроса карточек:", response.status_code, response.text)
        return None
def get_card_data(card_id):
    url = f"{BASE_URL}/cards/{card_id}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Ошибка запроса данных карточки {card_id}:", response.status_code, response.text)
        return None
def extract_email(text):
    if not text:
        return "Нет email"

    match = re.search(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", text)
    return match.group(0) if match else "Нет email"


SPACE_ID = 17503
TAG = "ДПК"

cards = get_cards_by_tag(SPACE_ID, TAG)

if cards:
    with open("cards_data_1.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Ссылка", "Email", "Описание", "Дата создания"])
        for card in cards:
            card_id = card.get("id")
            card_data = get_card_data(card_id)

            if card_data:
                link = f"https://proctoredu.kaiten.ru/card/{card_id}"
                description = card_data.get("description") or "Нет описания"
                email = extract_email(description)
                created_date = card_data.get("created", "Неизвестно")

                writer.writerow([link, email, description, created_date])
    print("Данные сохранены в 'cards_data.csv'")

Данные сохранены в 'cards_data.csv'


В коде ниже ищем по названию, а не по тегу - но по названию ищет плохо

In [ ]:
import requests
import csv
import re
import os



API_KEY = "your_api_key"
BASE_URL = "https://proctoredu.kaiten.ru/api/latest"



headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Accept": "application/json"
}


def get_cards_by_title_and_date(space_id, title, created_after, limit=100):
    url = f"{BASE_URL}/cards?space_id={space_id}&title={title}&created_after={created_after}&limit={limit}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print("Ошибка запроса карточек:", response.status_code, response.text)
        return None

def get_card_data(card_id):
    url = f"{BASE_URL}/cards/{card_id}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Ошибка запроса данных карточки {card_id}:", response.status_code, response.text)
        return None

def extract_email(text):
    if not text:
        return "Нет email"

    match = re.search(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", text)
    return match.group(0) if match else "Нет email"


SPACE_ID = 17503
TITLE = "Обработать запрос от нового клиента"
CREATED_AFTER = "2024-01-01T00:00:00Z"


cards = get_cards_by_title_and_date(SPACE_ID, TITLE, CREATED_AFTER)


if cards:
    file_exists = os.path.isfile("cards_data.csv")
    with open("cards_data.csv", "a", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)

        if not file_exists:
            writer.writerow(["Ссылка", "Email", "Описание", "Дата создания"])

        for card in cards:
            card_id = card.get("id")
            card_data = get_card_data(card_id)

            if card_data:
                link = f"https://proctoredu.kaiten.ru/card/{card_id}"
                description = card_data.get("description") or "Нет описания"
                email = extract_email(description)
                created_date = card_data.get("created", "Неизвестно")

                writer.writerow([link, email, description, created_date])

    print("Новые карточки добавлены в 'cards_data.csv'")

Новые карточки добавлены в 'cards_data.csv'
